In [ ]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image
from tqdm import tqdm
from efficientdet.constants import PROJECT_PATH

# Load data

In [ ]:
(Xtrain, ytrain), (Xtest, ytest) = tf.keras.datasets.mnist.load_data()
Xtrain.shape, ytrain.shape, Xtest.shape, ytest.shape

# Convert MNIST data to bounding box data

In [ ]:
# example image
plt.imshow(Xtrain[0], cmap='gray');

In [ ]:
def draw_mnist_to_image(img_mnist, h=512, w=512):
    # extract only relevant pixels
    xs, ys = np.where(img_mnist > 0)
    img_mnist_resized = img_mnist[xs.min()-1:xs.max()+2, ys.min()-1:ys.max()+2]
    # resize mnist image
    size = np.random.randint(75, 150)
    try:
        img_mnist_resized = tf.image.resize(np.expand_dims(img_mnist_resized, axis=-1), (size, size), preserve_aspect_ratio=True)
    except:
        pass
    img_mnist_resized = np.squeeze(img_mnist_resized)
    height, width = img_mnist_resized.shape[:2]
    # add mnist image to bigger image
    full_image = np.zeros((h, w), dtype=np.uint8)
    x1 = np.random.randint(0, w - width)
    y1 = np.random.randint(0, h - height)
    x2 = x1 + width
    y2 = y1 + height
    full_image[y1:y2, x1:x2] = img_mnist_resized
    return full_image, x1, y1, x2, y2


def mnist_to_bbox_data(xdata, store_dir):
    image_paths = []
    coordinates = []
    os.makedirs(store_dir, exist_ok=True)
    for i, img in tqdm(enumerate(xdata), total=len(xdata)):
        new_img, x1, y1, x2, y2 = draw_mnist_to_image(img)
        pil_img = Image.fromarray(new_img, mode='L')
        output_path = os.path.join(store_dir, "img{:05d}.jpg".format(i))
        pil_img.save(output_path)
        image_paths.append(output_path)
        coordinates.append([x1,y1,x2,y2])
    coordinates = np.array(coordinates, dtype=np.int16)
    return image_paths, coordinates


def create_data_df(image_paths, coordinates, labels):
    return pd.concat([
        pd.DataFrame(data={'img_path': image_paths}),
        pd.DataFrame(data=coordinates, columns=['x1', 'y1', 'x2', 'y2']),
        pd.DataFrame(data={'label': labels})
    ], axis=1)

In [ ]:
train_image_paths, train_coordinates = mnist_to_bbox_data(Xtrain, os.path.join(PROJECT_PATH, 'data/images_train'))
len(train_image_paths), train_coordinates.shape

In [ ]:
test_image_paths, test_coordinates = mnist_to_bbox_data(Xtest, os.path.join(PROJECT_PATH, 'data/images_test'))
len(test_image_paths), test_coordinates.shape

In [ ]:
df_train = create_data_df(train_image_paths, train_coordinates, ytrain)
df_train.head(2)

In [ ]:
df_train.to_csv(os.path.join(PROJECT_PATH, 'data/train.csv'), index=False)

In [ ]:
df_test = create_data_df(test_image_paths, test_coordinates, ytest)
df_test.head(2)

In [ ]:
df_test.to_csv(os.path.join(PROJECT_PATH, 'data/test.csv'), index=False)

# Visualize processed data

In [ ]:
def plot_processed_data(df, n_rows=4, n_cols=4):
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(4 * n_cols, 4 * n_rows))

    for i, ax in enumerate(np.ravel(axes)):
        row = df.iloc[i]
        ax.imshow(plt.imread(row['img_path']))
        x1, y1, x2, y2 = row['x1'], row['y1'], row['x2'], row['y2']
        rect = Rectangle((x1, y1), x2 - x1, y2 - y1, ec='red', fc='None')
        ax.add_patch(rect)
        ax.set_title("Label: {}".format(row['label']), fontsize=18)

In [ ]:
# training data
plot_processed_data(df_train)

In [ ]:
# test data
plot_processed_data(df_test, n_rows=2)